# Debiasing Agent


In [2]:
from debiasing.llm.models import OpenAICompletion
from debiasing.llm.models import AntrophicCompletion
from debiasing.llm.models import LLMMessage

In [4]:
openai = OpenAICompletion()

msgs = [
    LLMMessage(
        role=LLMMessage.MessageRole.USER,
        content="What is the capital of France?"
    )
]

text, response = openai.get_answer(
    messages=msgs
)

print(text)

The capital of France is Paris.


In [ ]:
# Ref: Antrophic models https://docs.anthropic.com/en/docs/about-claude/models
antrophic = AntrophicCompletion(model_id = 'claude-3-opus-20240229')

msgs = [
    LLMMessage(
        role=LLMMessage.MessageRole.USER,
        content="What is the capital of Chile?"
    )
]

text, response = antrophic.get_answer(
    messages=msgs
)

print(text)

The capital of Chile is Santiago.
